Note: This notebook is inspired from https://www.kaggle.com/fabiendaniel/customer-segmentation

With our data ready to be clustered from the notebook HackathonEDA. We can start clustering the data.

Let's first load the latest data

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', 50)

data = pd.read_csv("../../data/processed/tempDataFrameLandmark/ratefinalData2ndCheckpoint.csv")
data = data.drop(['Unnamed: 0'], axis = 1)
data.head()

### Creating clusters of counties with high number of tourist per population

In this section I will group the counties based on how popular they are in tourism considering their overall population. We will start with good old <b>kmeans</b> clustering. But before that, lets create a new dataframe that contains the values of tourism out of the complete data that has been prepared. We are interested in following fields. 

* lodgingInventoryBucketNonVacationalRental
* lodgingInventoryBucketVacationalRental
* lIBNonVRRatioToPopulation
* lIBVRRatioToPopulation
* aIPBRatioToPopulation
* AERAFEmployementRatio

In [ ]:
tourismByCounty = data.filter(['lodgingInventoryBucketNonVacationalRental', 'lodgingInventoryBucketVacationalRental', 'lIBNonVRRatioToPopulation', 'lIBVRRatioToPopulation', 'aIPBRatioToPopulation', 'AERAFEmployementRatio'], axis=1)
tourismByCounty.head()

In order to define (approximately) the number of clusters that best represents the data, I use the <b>silhouette score</b>:

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from tqdm import tqdm

matrix = tourismByCounty.as_matrix()
for n_clusters in tqdm(range(3,10)):
    kmeans = KMeans(init='k-means++', n_clusters = n_clusters, n_init=30)
    kmeans.fit(matrix)
    clusters = kmeans.predict(matrix)
    silhouette_avg = silhouette_score(matrix, clusters)
    print("For n_clusters =", n_clusters, "The average silhouette_score is :", silhouette_avg)
    if(n_clusters % 10 == 0):
        unique, counts = np.unique(clusters, return_counts=True)
        print("Minimum values in cluster is: " , min(counts))

The score for 3 clusters is best but this will be very less of a cluster. Let's group the values in 5 clusters.

In [ ]:
n_clusters = 5
silhouette_avg = -1
while silhouette_avg < 0.32:
    kmeans = KMeans(init='k-means++', n_clusters = n_clusters, n_init=30)
    kmeans.fit(matrix)
    clusters = kmeans.predict(matrix)
    silhouette_avg = silhouette_score(matrix, clusters)
    
    print("For n_clusters =", n_clusters, "The average silhouette_score is :", silhouette_avg)

<b>Characterizing the content of clusters</b>

Number of elements in each clusters

In [ ]:
unique, counts = np.unique(clusters, return_counts=True)
print(np.asarray((unique, counts)).T)

<b>Silhouette intra-cluster score</b>

In order to have an insight on the quality of the classification, we can represent the silhouette scores of each element of the different clusters.

In [ ]:
def graph_component_silhouette(n_clusters, lim_x, mat_size, sample_silhouette_values, clusters):
    plt.rcParams["patch.force_edgecolor"] = True
    plt.style.use('fivethirtyeight')
    mpl.rc('patch', edgecolor = 'dimgray', linewidth=1)
    #____________________________
    fig, ax1 = plt.subplots(1, 1)
    fig.set_size_inches(10, 8)
    ax1.set_xlim([lim_x[0], lim_x[1]])
    ax1.set_ylim([0, mat_size + (n_clusters + 1) * 10])
    y_lower = 10
    for i in range(n_clusters):
        #___________________________________________________________________________________
        # Aggregate the silhouette scores for samples belonging to cluster i, and sort them
        ith_cluster_silhouette_values = sample_silhouette_values[clusters == i]
        ith_cluster_silhouette_values.sort()
        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i
        cmap = cm.get_cmap("Spectral")
        color = cmap(float(i) / n_clusters)        
        ax1.fill_betweenx(np.arange(y_lower, y_upper), 0, ith_cluster_silhouette_values,
                           facecolor=color, edgecolor=color, alpha=0.8)
        #____________________________________________________________________
        # Label the silhouette plots with their cluster numbers at the middle
        ax1.text(-0.03, y_lower + 0.5 * size_cluster_i, str(i), color = 'red', fontweight = 'bold',
                bbox=dict(facecolor='white', edgecolor='black', boxstyle='round, pad=0.3'))
        #______________________________________
        # Compute the new y_lower for next plot
        y_lower = y_upper + 10

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.cm as cm

#____________________________________
# define individual silouhette scores
sample_silhouette_values = silhouette_samples(matrix, clusters)
#__________________
# and do the graph
graph_component_silhouette(n_clusters, [-0.4, 0.7], len(tourismByCounty), sample_silhouette_values, clusters)

Now we can take a look at what each cluster and what is in them. And Rank clusters which attract more tourist per population.

In [ ]:
import operator
clusterScore = {}

for i in range(n_clusters):
    county_cluster = tourismByCounty.loc[clusters == i]
    clusterScore[i] = (county_cluster['lIBNonVRRatioToPopulation'].sum() + 
                           county_cluster['lIBVRRatioToPopulation'].sum() + 
                           county_cluster['aIPBRatioToPopulation'].sum() + 
                           county_cluster['AERAFEmployementRatio'].sum()) / len(county_cluster)
    
sorted_d = dict( sorted(clusterScore.items(), key=operator.itemgetter(1),reverse=True))
print('Dictionary in descending order by value : ',sorted_d)

Let's add the cluster number to the counties in main dataframe

In [ ]:
data['tourismCluster'] = 0
for i in tqdm(range(n_clusters)):
    county_cluster = tourismByCounty.loc[clusters == i]
    for index, value in county_cluster['AERAFEmployementRatio'].items():
        data['tourismCluster'][index] = i 

Let's look at the first county with highest tourism per population i.e counties in cluster 2.

In [ ]:
data[data['tourismCluster'] == 2].iloc[0]

This is the county: Denali Borough in Alaska. And Denali National Park is Alaska’s most popular land attraction. Source: https://www.alaska.org/destination/denali-national-park

Which proves that clustering done by our algorithm is quite nice. Here is a image of this beautiful county.

In [ ]:
from IPython.display import Image
Image("../../data/raw/Images/denali.jpg")

Adding a checkpoint

In [ ]:
data.to_csv(r'../../data/processed/tempDataFrameLandmark/dataWithTourismCluster.csv') 